In [17]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Path ke folder test dan model yang telah dilatih
test_dir = "data/original_dataset_utama/test2/"
model_path = "efficientnet_b7_finetuned.h5"  # Ganti dengan path ke model yang telah dilatih

# Load model yang telah dilatih
model = load_model(model_path)

# Menggunakan ImageDataGenerator untuk data test
test_datagen = ImageDataGenerator(rescale=1./255)

# Pastikan struktur folder test memiliki sub-folder 'images' untuk memuat seluruh gambar
# test_generator akan mencari gambar di `test/images/`
test_generator = test_datagen.flow_from_directory(
    directory=os.path.dirname(test_dir),  # `os.path.dirname(test_dir)` untuk ke direktori `test`
    target_size=(224, 224),  # Sesuaikan dengan input shape model
    batch_size=1,  # Gunakan batch_size = 1 untuk memprediksi satu gambar pada satu waktu
    class_mode=None,  # Tidak ada label untuk data test
    shuffle=False,  # Tidak diacak, agar urutan file sesuai dengan urutan prediksi
)

print(f"Jumlah file yang ditemukan: {len(test_generator.filenames)}")

# Melakukan prediksi pada data test
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)

# Mengambil nama file dari generator
file_names = test_generator.filenames

# Menghapus prefix `images/` dari nama file (misal `images/image_1.jpg` -> `image_1.jpg`)
file_names = [os.path.basename(name) for name in file_names]

# Buat `image_id` menjadi format `test_0`, `test_1`, `test_2`, dst.
# Gunakan `enumerate` untuk menambahkan nomor urut ke dalam image_id
image_ids = [f"test_{i}" for i, _ in enumerate(file_names)]

# Membuat dataframe dari hasil prediksi
# `predictions` adalah probabilitas untuk setiap kelas. Asumsikan urutan kelas [healthy, multiple_disease, rust, scab]
submission_df = pd.DataFrame(predictions, columns=['healthy', 'multiple_diseases', 'rust', 'scab'])

# Menambahkan kolom `image_id` ke dataframe sesuai urutan
submission_df['image_id'] = image_ids

# Menyusun ulang kolom agar `image_id` berada di depan
submission_df = submission_df[['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab']]

# Menyimpan hasil ke file CSV dengan format yang sesuai
submission_df.to_csv('submission.csv', index=False)

print("File submission.csv berhasil dibuat dengan format image_id seperti 'test_0', 'test_1', 'test_2', ...")

Found 1821 images belonging to 1 classes.
Jumlah file yang ditemukan: 1821
1821/1821 [==============================] - 43s 22ms/step
File submission.csv berhasil dibuat dengan format image_id seperti 'test_0', 'test_1', 'test_2', ...


In [4]:
import os

# Path ke folder yang ingin dihitung jumlah filenya
folder_path = 'data/original_dataset_utama/test2/images'

# Menghitung jumlah file di dalam folder
num_files = len([file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))])

print(f"Jumlah file di dalam folder '{folder_path}': {num_files}")


Jumlah file di dalam folder 'data/original_dataset_utama/test2/images': 1821


In [13]:
import os

# Path ke folder test/images
folder_path = 'data/original_dataset_utama/test2/'

# Mengambil semua item di dalam folder test/images
items_in_folder = os.listdir(folder_path)

# Mengecek apakah ada sub-folder di dalam test/images
sub_folders = [item for item in items_in_folder if os.path.isdir(os.path.join(folder_path, item))]

if sub_folders:
    print(f"Terdapat sub-folder di dalam {folder_path}: {sub_folders}")
else:
    print(f"Tidak ada sub-folder di dalam {folder_path}. Semua gambar berada di dalam folder ini.")


Terdapat sub-folder di dalam data/original_dataset_utama/test2/: ['images']


In [10]:
rm -r data/original_dataset_utama/test2/.ipynb_checkpoints